In [55]:
from abc import ABCMeta
from typing import Union, Iterable, Callable\

from itertools import combinations

from qutip import *
from qutip.qip.operations import *
from qutip.qip.circuit import QubitCircuit, Gate

import numpy as np
from numpy.typing import ArrayLike
from numpy import pi

import scipy as sp
import scipy.integrate as spint
import scipy.linalg as splinalg
import scipy.optimize as spopt

import plotly.express as px

import matplotlib.pyplot as plt

Implemet simple ´qaoa´ solver for general Hamiltonian given as $Z$-polynomial

$$

    H = C + \sum_{i}c_{i}Z_{i} + \sum_{i,j}c_{i,j}Z_{i}Z_{j} + \sum_{i,j,k}c_{i,j,k}Z_{i}Z_{j}Z_{k} + \dots

$$
\\
For that write global functions for all parts of `qaoa` $\rightarrow$ *trial state preperation*, *measurement*, computing the *expectation* .
Write custom, abstract `OPTIMIZER` class wich can be substituted by scipy optimizers like `cobyla` and then later by the `tdvp optimizer`. 

In [54]:
def sz(n:int,i:int):
    return expand_operator(sigmaz(),n,i,[2 for _ in range(n)])

def sx(n:int,i:int):
    return expand_operator(sigmax(),n,i,[2 for _ in range(n)])
    
H = tensor(sigmaz(),sigmaz(),sigmaz())
B = sum([sx(3,_) for _ in range(3)])
minus = (basis(2,0)-basis(2,1)).unit()
minus_minus = tensor(minus,minus)

def rzz(arg_value):
    return tensor(rz(arg_value),rz(-arg_value))

In [5]:
B

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0. 1. 1. 0. 1. 0. 0. 0.]
 [1. 0. 0. 1. 0. 1. 0. 0.]
 [1. 0. 0. 1. 0. 0. 1. 0.]
 [0. 1. 1. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 1. 1. 0.]
 [0. 1. 0. 0. 1. 0. 0. 1.]
 [0. 0. 1. 0. 1. 0. 0. 1.]
 [0. 0. 0. 1. 0. 1. 1. 0.]]

In [142]:
def qaoa_circuit(p: int,  betas: list[float], gammas: list[float],
                 hamiltonian: Union[ArrayLike, Qobj] = None,
                 qubo:ArrayLike=None, linears:ArrayLike=None, quadratics:ArrayLike=None,
                 initial_state: Qobj=minus_minus, mixer: Qobj = None) -> QubitCircuit:
    # initiailize mixer operator
    n = 0

    # check input mode, prefere qubo mode
    if linears is not None and quadratics is not None:
        linears = np.array(linears)
        quadratics = np.array(quadratics)
        qubo = quadratics.copy()
        linears += quadratics.diagonal()
        np.fill_diagonal(qubo,linears)
    # enter if either qubo input or linears and quadratics were given
    if qubo is not None:
        qubo = np.array(qubo)
        n = qubo.shape[0]
        print(n)
        # define what to apply to the circuit in each H_p turn
        def qcH(gamma:float):
            qc = QubitCircuit(n)
            qc.user_gates = {"RZZ",rzz}
            for j in range(n):
                print(j)
                qc.add_gate("RZ",targets=j, arg_value=2*gamma*qubo[j][j])
            for j,k in combinations(range(n),2):
                qc.add_gate("RZZ", targets=[j,k], arg_value=2*gamma*qubo[j][k])
            return qc
    # if no qubo data is given but a Qobj hamiltonian, exponiate it
    elif hamiltonian is not None:
        assert hamiltonian.isherm, "hamiltonian must be hermetian"
        n = np.log2(hamiltonian.dims[0][0])
        def qcH(gamma:float):
            def H_exp(arg_value):
                return (-1j*arg_value*H).expm()
            qc = QubitCircuit(n)
            qc.user_gates = {"H_exp":H_exp}
            qc.add_gate("H_exp", arg_value=gamma)

    assert n>0
    assert qcH is not None

    # define mixer circuit
    if mixer is None:
        initial_state_list = ['+' for _ in range(n)]
        def qcB(beta:float):
            qc = QubitCircuit(n)
            qc.add_1q_gate("RX",arg_value=2*beta)
            return qc
    else:
        initial_state_list = None
        print("Computing groundstate of mixer")
        initial_state = mixer.groundstate()[1]
        def B_exp(arg_value):
                return (-1j*arg_value*mixer).expm()
        def qcB(beta:float):
            qc = QubitCircuit(n)
            qc.user_gates = {"B_exp":B_exp}
            qc.add_gate("B_exp", arg_value=beta)


    qc = QubitCircuit(n, input_states=initial_state_list)

    for i in range(p):
        qc.add_circuit(qcH(gammas[i]))
        qc.add_circuit(qcB(betas[i]))

    return qc


In [144]:
qaoa = qaoa_circuit(2,[.1,.2],[.3,.4],qubo=np.array([[1,2],[0,3]]))

2
0
1
0
1


In [8]:
def expectation(circuit, function) -> float:
    pass

In [ ]:
class OPTIMIZER(metaclass=ABCMeta):
    pass